# Assignment 3

Assignment 3 Nov 1 2200h

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [3]:
# Load libraries
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import numpy as np
import shap


In [15]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()

# Define non-categorical features (numerical features)
non_categorical_features = [
    'coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'
]

# Separate the non-categorical features
X_non_categorical = fires_dt[non_categorical_features]

# Check for missing values
if X_non_categorical.isnull().sum().any():
    print("Missing values found in non-categorical features. Handling missing values...")
    # Impute missing values for numeric columns
    imputer = SimpleImputer(strategy='mean')  # You can choose other strategies like 'median' or 'most_frequent'
    X_non_categorical = pd.DataFrame(imputer.fit_transform(X_non_categorical), columns=non_categorical_features)
else:
    print("No missing values found in non-categorical features.")

# Scale the features
scaler = StandardScaler()
X_non_categorical_scaled = pd.DataFrame(scaler.fit_transform(X_non_categorical), columns=non_categorical_features)

# Display the preprocessed non-categorical features
print("Preprocessed Non-Categorical Features:")
print(X_non_categorical_scaled.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB
No missing values found in non-categorical features.
Preprocessed Non-Categorical Features:
    coord_x   coord_y      ffmc       dmc        dc       isi      temp  \
0  1.008313  0.569860 -0.805959 -1.323326 -1.830477 -0.860946 -1.842640   
1  1.008313 -0.24400

# Get X and Y

Create the features data frame and target data.

In [16]:
# Get X and Y
X = fires_dt.drop(columns=['area'])  # Features: all columns except 'area'
y = fires_dt['area']  # Target: 'area'

# Print features of DataFrame(X)
print("Features (X):")
print(X.head())  # Display the first few rows of X

# Print target data (y)
print("\nTarget (y):")
print(y.head())  # Display the first few rows of y

Features (X):
   coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain
0        7        5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0
1        7        4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0
2        7        4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0
3        8        6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2
4        8        6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0

Target (y):
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: area, dtype: float64


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [17]:
# Preproc 1 Numeric variables scaled, categorical variables one-hot encoded
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']),
        ('cat', OneHotEncoder(), ['month', 'day'])
    ]
)

# Print Preproc 1
print("Preproc 1:")
print(preproc1)

Preproc 1:
ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat', OneHotEncoder(), ['month', 'day'])])


### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [18]:
# Preproc 2 Numeric variables scaled with a non-linear transformation, categorical variables one-hot encoded
from sklearn.preprocessing import FunctionTransformer
import numpy as np

preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler()),
            ('nonlinear_transform', FunctionTransformer(np.log1p, validate=True))   
        ]), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']),
        ('cat', OneHotEncoder(), ['month', 'day'])
    ]
)

# Print Preproc 2
print("\nPreproc 2:")
print(preproc2)


Preproc 2:
ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler()),
                                                 ('nonlinear_transform',
                                                  FunctionTransformer(func=<ufunc 'log1p'>,
                                                                      validate=True))]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('cat', OneHotEncoder(), ['month', 'day'])])


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [19]:
# Pipeline A = preproc1 + baseline (Lasso)
pipeline_a = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', Lasso())
])

# Print Pipeline A
print("Pipeline A:")
print(pipeline_a)

Pipeline A:
Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat', OneHotEncoder(),
                                                  ['month', 'day'])])),
                ('regressor', Lasso())])


In [20]:
# Pipeline B = preproc2 + baseline (Lasso)
pipeline_b = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', Lasso())
])

# Print Pipeline B
print("\nPipeline B:")
print(pipeline_b)


Pipeline B:
Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('nonlinear_transform',
                                                                   FunctionTransformer(func=<ufunc 'log1p'>,
                                                                                       validate=True))]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat', OneHotEncoder(),
                                                  ['month', 'day'])])),
                ('regressor', Lasso())])


In [21]:
# Pipeline C = preproc1 + advanced model (RandomForestRegressor)
pipeline_c = Pipeline(steps=[
    ('preprocessing', preproc1),
    ('regressor', RandomForestRegressor())
])

# Print Pipeline C
print("\nPipeline C:")
print(pipeline_c)


Pipeline C:
Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat', OneHotEncoder(),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor())])


In [22]:
# Pipeline D = preproc2 + advanced model (RandomForestRegressor)
pipeline_d = Pipeline(steps=[
    ('preprocessing', preproc2),
    ('regressor', RandomForestRegressor())
])

# Print Pipeline D
print("\nPipeline D:")
print(pipeline_d)


Pipeline D:
Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('nonlinear_transform',
                                                                   FunctionTransformer(func=<ufunc 'log1p'>,
                                                                                       validate=True))]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('cat', OneHotEncoder(),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegre

In [24]:
# Check for missing values in the dataset
missing_values = fires_dt.isnull().sum()
print("Missing values in each column:")
print(missing_values[missing_values > 0])

Missing values in each column:
Series([], dtype: int64)


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [25]:
# Check for missing values
if X.isnull().sum().any() or y.isnull().sum() > 0:
    print("Missing values found in the dataset. Handling missing values...")
    # Impute missing values for numeric columns
    imputer = SimpleImputer(strategy='mean')
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
else:
    print("No missing values found.")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define preprocessors
preproc1 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
            ('scaler', StandardScaler())
        ]), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']),
        ('cat', OneHotEncoder(), ['month', 'day'])
    ]
)

preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
            ('scaler', StandardScaler()),
            ('nonlinear_transform', FunctionTransformer(np.log1p, validate=True))
        ]), ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']),
        ('cat', OneHotEncoder(), ['month', 'day'])
    ]
)

# Define pipelines
pipeline_a = Pipeline(steps=[('preprocessing', preproc1), ('regressor', Lasso())])
pipeline_b = Pipeline(steps=[('preprocessing', preproc2), ('regressor', Lasso())])
pipeline_c = Pipeline(steps=[('preprocessing', preproc1), ('regressor', RandomForestRegressor())])
pipeline_d = Pipeline(steps=[('preprocessing', preproc2), ('regressor', RandomForestRegressor())])

# Define parameter grids for each pipeline
param_grid_a = {'regressor__alpha': [0.1, 1.0, 10.0, 100.0]}  # For Lasso in Pipeline A
param_grid_b = {'regressor__alpha': [0.1, 1.0, 10.0, 100.0]}  # For Lasso in Pipeline B
param_grid_c = {'regressor__n_estimators': [50, 100, 150, 200]}  # For RandomForestRegressor in Pipeline C
param_grid_d = {'regressor__n_estimators': [50, 100, 150, 200]}  # For RandomForestRegressor in Pipeline D

# Perform GridSearch on each of the four pipelines
try:
    grid_a = GridSearchCV(pipeline_a, param_grid_a, cv=5, n_jobs=-1, verbose=1)
    grid_a.fit(X_train, y_train)
    print("Best parameters for Pipeline A:", grid_a.best_params_)
    print("Best score for Pipeline A:", grid_a.best_score_)
except Exception as e:
    print("Error in GridSearch for Pipeline A:", e)

try:
    grid_b = GridSearchCV(pipeline_b, param_grid_b, cv=5, n_jobs=-1, verbose=1)
    grid_b.fit(X_train, y_train)
    print("\nBest parameters for Pipeline B:", grid_b.best_params_)
    print("Best score for Pipeline B:", grid_b.best_score_)
except Exception as e:
    print("Error in GridSearch for Pipeline B:", e)

try:
    grid_c = GridSearchCV(pipeline_c, param_grid_c, cv=5, n_jobs=-1, verbose=1)
    grid_c.fit(X_train, y_train)
    print("\nBest parameters for Pipeline C:", grid_c.best_params_)
    print("Best score for Pipeline C:", grid_c.best_score_)
except Exception as e:
    print("Error in GridSearch for Pipeline C:", e)

try:
    grid_d = GridSearchCV(pipeline_d, param_grid_d, cv=5, n_jobs=-1, verbose=1)
    grid_d.fit(X_train, y_train)
    print("\nBest parameters for Pipeline D:", grid_d.best_params_)
    print("Best score for Pipeline D:", grid_d.best_score_)
except Exception as e:
    print("Error in GridSearch for Pipeline D:", e)

No missing values found.
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters for Pipeline A: {'regressor__alpha': 0.1}
Best score for Pipeline A: nan
Fitting 5 folds for each of 4 candidates, totalling 20 fits


One or more of the test scores are non-finite: [nan nan nan nan]


Error in GridSearch for Pipeline B: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "c:\Users\Q\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\base.py", line 1473, in wra

One or more of the test scores are non-finite: [nan nan nan nan]



Best parameters for Pipeline C: {'regressor__n_estimators': 50}
Best score for Pipeline C: nan
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Best parameters for Pipeline D: {'regressor__n_estimators': 50}
Best score for Pipeline D: nan


One or more of the test scores are non-finite: [nan nan nan nan]
invalid value encountered in log1p


# Evaluate

+ Which model has the best performance?

Unfortunately, due to the problems with my code in my previous sections, I am unable to determine which model has the best performance. 

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.